In [461]:
del list

In [46]:
# Tian 1:10, 17:26
subcortical_index = list(range(0,10)) + list(range(16,26))
subcortical_name = ['aHIP-rh', 'pHIP-rh', 'lAMY-rh', 'mAMY-rh', 'THA-DP-rh', 'THA-VP-rh', 'THA-VA-rh', 'THA-DA-rh', 'NAc-shell-rh', 'NAc-core-rh', 'aHIP-lh', 'pHIP-lh', 'lAMY-lh', 'mAMY-lh', 'THA-DP-lh', 'THA-VP-lh', 'THA-VA-lh', 'THA-DA-lh', 'NAc-shell-lh', 'NAc-core-lh']
# Schaefer: 
# lh-mPFC: 199:205
# rh-mPFC: 464:470
# lh-Ins: 67, 108:111, 126:128
# rh-Ins: 319, 361:364, 383:386
## ACC: 390
# Glasser
cortical_roi = ['lh_dlPFC', 'rh_dlPFC', 'lh_mPFC', 'rh_mPFC', 'lh_PCC', 'rh_PCC', 'lh_Ins', 'rh_Ins']
lh_dlPFC_index = [205, 246, 247, 249, 250, 252, 262, 263, 264, 265, 266, 276, 277]
rh_dlPFC_index = [25, 66, 67, 69, 70, 72, 82, 83, 84, 85, 86, 96, 97]
lh_mPFC_index = [236, 237, 238, 239, 240, 241, 242, 243, 244, 248, 267, 343, 344, 345, 358, 359]
rh_mPFC_index = [56, 57, 58, 59, 60, 61, 62, 63, 64, 68, 87, 163, 164, 165, 178, 179]
lh_PCC_index = [193, 194, 206, 209, 210, 211, 212, 213, 214, 300, 321, 340, 341]
rh_PCC_index = [13, 14, 26, 29, 30, 31, 32, 33, 34, 120, 141, 160, 161]
lh_Ins_index = [285, 287, 288, 289, 290, 291, 293, 294, 346, 347, 348, 357]
rh_Ins_index = [105, 107, 108, 109, 110, 111, 113, 114, 166, 167, 168, 177]
dic_cortical_roi = {
    'lh_dlPFC': lh_dlPFC_index,
    'rh_dlPFC': rh_dlPFC_index,
    'lh_mPFC': lh_mPFC_index,
    'rh_mPFC': rh_mPFC_index,
    'lh_PCC': lh_PCC_index,
    'rh_PCC': rh_PCC_index,
    'lh_Ins': lh_Ins_index,
    'rh_Ins': rh_Ins_index
}

In [49]:
import itertools
roi_list = cortical_roi + subcortical_name
print(roi_list)
connectivity_labels = [
    f"{roi1}--{roi2}" for roi1, roi2 in itertools.combinations(roi_list, 2)
]

# Display the first 10 connectivity labels as a sample
print(connectivity_labels[:10])

['lh_dlPFC', 'rh_dlPFC', 'lh_mPFC', 'rh_mPFC', 'lh_PCC', 'rh_PCC', 'lh_Ins', 'rh_Ins', 'aHIP-rh', 'pHIP-rh', 'lAMY-rh', 'mAMY-rh', 'THA-DP-rh', 'THA-VP-rh', 'THA-VA-rh', 'THA-DA-rh', 'NAc-shell-rh', 'NAc-core-rh', 'aHIP-lh', 'pHIP-lh', 'lAMY-lh', 'mAMY-lh', 'THA-DP-lh', 'THA-VP-lh', 'THA-VA-lh', 'THA-DA-lh', 'NAc-shell-lh', 'NAc-core-lh']
['lh_dlPFC--rh_dlPFC', 'lh_dlPFC--lh_mPFC', 'lh_dlPFC--rh_mPFC', 'lh_dlPFC--lh_PCC', 'lh_dlPFC--rh_PCC', 'lh_dlPFC--lh_Ins', 'lh_dlPFC--rh_Ins', 'lh_dlPFC--aHIP-rh', 'lh_dlPFC--pHIP-rh', 'lh_dlPFC--lAMY-rh']


In [50]:
len(connectivity_labels)

378

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure


# === Step 1: Define Functions === #

def load_dataset(base_dir, data_set):
    """
    Load dataset-specific files.
    """
    fMRIinfo_file_path = os.path.join(base_dir, f"{data_set}_data_set.csv")
    participant_file_path = os.path.join(base_dir, "participants_fMRI.csv")
    return pd.read_csv(fMRIinfo_file_path), pd.read_csv(participant_file_path)


def load_subject_timeseries(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index):
    """
    Load cortical and subcortical timeseries data for a subject.
    """
    cortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Glasser.csv.gz"
    cortical_file_path = os.path.join(derivatives_dir, "timeseries", cortical_file_name)

    subcortical_file_name = f"sub-{subject_ID}_ses-{session_ID}_task-rest_space-Tian_Subcortex_S2_3T.csv.gz"
    subcortical_file_path = os.path.join(derivatives_dir, "timeseries", subcortical_file_name)

    if not (os.path.exists(cortical_file_path) and os.path.exists(subcortical_file_path)):
        print(f"Missing files for subject {subject_ID}, session {session_ID}.")
        return None

    # Load and process cortical timeseries
    df_cortical_all = pd.read_csv(cortical_file_path, compression="gzip", index_col=0, header=0)
    df_cortical_roi = pd.DataFrame({
        roi: df_cortical_all.iloc[dic_cortical_roi[roi]].mean(axis=0)
        for roi in dic_cortical_roi.keys()
    })

    # Load and process subcortical timeseries
    df_subcortical_all = pd.read_csv(subcortical_file_path, compression="gzip", index_col=0, header=0)
    df_subcortical_roi = df_subcortical_all.iloc[subcortical_index]

    # Combine cortical and subcortical ROIs
    return pd.concat([df_cortical_roi, df_subcortical_roi.transpose()], axis=1)


def clean_data(data):
    """
    Handle missing values and remove constant features from time series data.
    """
    # Fill NaNs with column-wise means
    data_filled = np.copy(data)
    for j in range(data.shape[1]):
        if np.isnan(data[:, j]).any():
            data_filled[:, j] = np.nan_to_num(data[:, j], nan=np.nanmean(data[:, j]))

    # Remove constant features
    non_constant_features = data_filled[:, data_filled.std(axis=0) != 0]
    return non_constant_features


def compute_connectivity(data):
    """
    Compute connectivity matrix for the given time series data.
    """
    correlation_measure = ConnectivityMeasure(kind="correlation")
    return correlation_measure.fit_transform([data])[0]


def extract_upper_triangle(matrix):
    """
    Extract the upper triangle values (excluding diagonal) from a connectivity matrix.
    """
    upper_tri_indices = np.triu_indices(matrix.shape[0], k=1)
    return matrix[upper_tri_indices]


# === Step 2: Define the Pipeline === #

def process_fMRI_subject(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index):
    """
    Full pipeline for processing a single subject's fMRI data.
    """
    # Load subject timeseries
    df_roi = load_subject_timeseries(subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index)
    if df_roi is None:
        return None, None

    # Clean data
    #cleaned_data = clean_data(df_roi.values)

    # Standardize data
    standardized_data = StandardScaler().fit_transform(df_roi.values)

    # Compute connectivity matrix
    connectivity_matrix = compute_connectivity(standardized_data)

    # Extract upper triangle
    upper_triangle = extract_upper_triangle(connectivity_matrix)

    return upper_triangle, subject_ID


def process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index):
    """
    Full pipeline for processing fMRI data for all subjects.
    """
    # Set paths
    base_dir = os.path.join(user_dir, project_name, "data")
    derivatives_dir = os.path.join(base_dir, "derivatives")

    # Load dataset
    df_fMRIinfo, df_participants = load_dataset(base_dir, data_set)
    subject_IDs = df_fMRIinfo["eid"].unique()

    # Initialize lists for data
    connectivity_data = []
    subject_ids_cleaned = []

    # Process each subject individually
    for subject_ID in subject_IDs:
        upper_triangle, cleaned_subject_ID = process_fMRI_subject(
            subject_ID, session_ID, derivatives_dir, dic_cortical_roi, subcortical_index
        )
        if upper_triangle is not None:
            connectivity_data.append(upper_triangle)
            subject_ids_cleaned.append(cleaned_subject_ID)

    # Filter participants based on available data
    df_filtered = df_participants.loc[df_participants["eid"].isin(subject_ids_cleaned)]

    return np.array(connectivity_data), df_filtered


In [26]:
#codes for modeling
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
# Perform stratified 10-Fold Cross-Validation
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Ensure binary target
def ensure_binary_target(y):
    unique_values = np.unique(y)
    if len(unique_values) > 2:
        raise ValueError("Target variable contains more than two classes. Please preprocess the data.")
    if unique_values.dtype == bool:
        return y.astype(int)
    elif set(unique_values) == {0, 1} or set(unique_values) == {1, 0}:
        return y
    else:
        raise ValueError("Target variable is not binary. Please preprocess the data.")


from sklearn.model_selection import train_test_split
# Split data into training and testing sets while preserving class distribution
def split_data(X, y, N_random_state, test_size=0.2):
    """
    Split the data into training and testing sets while preserving class ratios.
    
    Parameters:
    - X: Features.
    - y: Target labels.
    - test_size: Proportion of the dataset to include in the test split.
    - random_state: Random state for reproducibility.
    
    Returns:
    - X_train, X_test, y_train, y_test
    """
    return train_test_split(X, y, test_size=test_size, random_state=N_random_state, stratify=y)



# Model selection using cross-validation
def model_selection(X_train, y_train):
    models = {
        "Logistic Regression": LogisticRegression(),  # Provides coefficients (coef_)
        "Ridge Classifier": LogisticRegression(penalty='l2', solver='liblinear'),  # coef_
        "Lasso (L1)": LogisticRegression(penalty='l1', solver='liblinear'),  # coef_
        "LDA": LinearDiscriminantAnalysis(),  # Provides coefficients (coef_)
        "Perceptron": Perceptron(),  # Provides coefficients (coef_)
        "SVM (Linear)": SVC(kernel='linear'),  # Provides coefficients (coef_) when kernel='linear'
        "Random Forest": RandomForestClassifier(),  # Provides feature_importances_
    }

    best_model = None
    best_score = -np.inf
    best_name = ""

    for model_name, model in models.items():
        cv_score = cross_val_score(model, X_train, y_train, cv=stratified_kfold, scoring='accuracy').mean()
        print(f"Model: {model_name}, CV Score: {cv_score:.4f}")

        if cv_score > best_score:
            best_score = cv_score
            best_model = model
            best_name = model_name

    print(f"Best Model: {best_name} with CV score: {best_score:.4f}")
    return best_model, best_name


from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

def feature_selection_with_rfe(X_train, y_train, n_features, best_model):
    """
    Perform feature selection using RFE, with fallback to univariate selection for models without coefficients.
    """
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        # Use RFE for models with coefficients or feature importances
        rfe = RFE(estimator=best_model, n_features_to_select=n_features, step=1)
        rfe.fit(X_train, y_train)

        selected_features = rfe.support_
        if np.sum(selected_features) == 0:
            print("No features selected using RFE. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    else:
        # Fallback to univariate feature selection
        print("Model lacks coefficients/feature importance; using univariate feature selection.")
        
        # Use SelectKBest with F-statistic (or mutual information if preferred)
        selector = SelectKBest(score_func=f_classif, k=n_features)
        selector.fit(X_train, y_train)

        selected_features = selector.get_support()
        if np.sum(selected_features) == 0:
            print("No features selected using univariate method. Using all features as fallback.")
            selected_features = np.ones(X_train.shape[1], dtype=bool)

    return selected_features

from sklearn.feature_selection import RFE, SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import numpy as np

def feature_selection_with_rfe_cv(X_train, y_train, best_model, scoring_metric='accuracy'):
    """
    Perform feature selection using RFE or univariate selection, optimizing the number of features automatically
    using cross-validation.
    """
    
    def evaluate_features(model, X, y, num_features):
        """
        Helper function to evaluate the model's performance with the given number of features using cross-validation.
        """
        if hasattr(model, "coef_") or hasattr(model, "feature_importances_"):
            # Perform RFE with the given number of features
            rfe = RFE(estimator=model, n_features_to_select=num_features, step=1)
            X_selected = rfe.fit_transform(X, y)
        else:
            # Use univariate feature selection as a fallback
            selector = SelectKBest(score_func=f_classif, k=num_features)
            X_selected = selector.fit_transform(X, y)

        # Evaluate model performance using cross-validation
        scores = cross_val_score(model, X_selected, y, cv=stratified_kfold, scoring=scoring_metric)
        return scores.mean()

    # Iterate over a range of features to find the optimal number of features
    best_score = -np.inf
    optimal_num_features = 0
    for num_features in range(1, X_train.shape[1] + 1):
        score = evaluate_features(best_model, X_train, y_train, num_features)
        if score > best_score:
            best_score = score
            optimal_num_features = num_features

    print(f"Optimal number of features: {optimal_num_features} with cross-validated score: {best_score:.4f}")

    # Perform final RFE or univariate selection with the optimal number of features
    if hasattr(best_model, "coef_") or hasattr(best_model, "feature_importances_"):
        rfe = RFE(estimator=best_model, n_features_to_select=optimal_num_features, step=1)
        rfe.fit(X_train, y_train)
        selected_features = rfe.support_
    else:
        selector = SelectKBest(score_func=f_classif, k=optimal_num_features)
        selector.fit(X_train, y_train)
        selected_features = selector.get_support()

    return selected_features, optimal_num_features


# Two-step grid search for hyperparameter optimization
def tune_model_hyperparameters(model, model_name, X_train, y_train):
    refined_grid = {}  # Initialize with a default value to avoid "unbound variable" error

    if model_name == "Logistic Regression":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Ridge Classifier":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Lasso (L1)" or model_name == "ElasticNet (L1+L2)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "SVM (Linear)":
        broad_param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    elif model_name == "Random Forest":
        broad_param_grid = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
    elif model_name == "Perceptron":
        broad_param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}
    elif model_name == "LDA":
        broad_param_grid = {'shrinkage': [None, 'auto'], 'solver': ['svd', 'lsqr', 'eigen']}
    else:
        raise ValueError(f"Model {model_name} does not have a defined parameter grid.")

    # Broad Grid Search
    broad_search = GridSearchCV(model, broad_param_grid, cv=stratified_kfold, scoring='accuracy', verbose=1, n_jobs=-1)
    broad_search.fit(X_train, y_train)
    best_params_broad = broad_search.best_params_

    # Define refined grid based on broad search results
    if model_name in ["Logistic Regression", "Lasso (L1)", "ElasticNet (L1+L2)", "SVM (Linear)"]:
        refined_grid = {'C': np.linspace(best_params_broad['C'] * 0.1, best_params_broad['C'] * 10, 5)}
    elif model_name == "Random Forest":
        refined_grid = {
            'n_estimators': [max(10, best_params_broad['n_estimators'] - 50), best_params_broad['n_estimators'], best_params_broad['n_estimators'] + 50],
            'max_depth': [None] if not best_params_broad['max_depth'] else [
                max(1, best_params_broad['max_depth'] - 5), best_params_broad['max_depth'], best_params_broad['max_depth'] + 5]
        }
    elif model_name == "Perceptron":
        refined_grid = {'alpha': np.linspace(best_params_broad['alpha'] * 0.1, best_params_broad['alpha'] * 10, 5)}
    elif model_name == "LDA":
        refined_grid = {'shrinkage': [best_params_broad['shrinkage']], 'solver': [best_params_broad['solver']]}

    # Narrow Grid Search
    narrow_search = GridSearchCV(model, refined_grid, cv=stratified_kfold, scoring='accuracy', verbose=1, n_jobs=-1)
    narrow_search.fit(X_train, y_train)

    print(f"Best Parameters (Broad Search): {best_params_broad}")
    print(f"Best Parameters (Narrow Search): {narrow_search.best_params_}")

    best_model = narrow_search.best_estimator_
    return best_model


from sklearn.metrics import roc_auc_score
# Train and evaluate final model
def train_and_evaluate_final_model(X_train, y_train, X_test, y_test, model):
    """
    Train and evaluate the final model. Reports accuracy and AUC.
    Parameters:
    - X_train: Features for training.
    - y_train: Labels for training.
    - X_test: Features for testing.
    - y_test: Labels for testing.
    - model: Machine learning model (must support `fit` and `predict_proba`).

    Returns:
    - model: Trained model.
    - test_accuracy: Accuracy on the test set.
    - test_auc: AUC score on the test set.
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Evaluate accuracy
    test_accuracy = model.score(X_test, y_test)
    
    # Predict probabilities for AUC computation
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
        test_auc = roc_auc_score(y_test, y_proba)
        print(f"Test Set AUC (Final Model): {test_auc:.4f}")
    else:
        print("Model does not support probability predictions; skipping AUC computation.")
        test_auc = None

    print(f"Test Set Accuracy (Final Model): {test_accuracy:.4f}")
    y_pred = model.predict(X_test)
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    return model, test_accuracy, test_auc



# Cosine similarity between two models
from sklearn.metrics.pairwise import cosine_similarity

def calculate_model_similarity(model1, model2):
    """
    Calculate the similarity between two models using cosine similarity or feature importances.

    Parameters:
    - model1: First trained model
    - model2: Second trained model

    Returns:
    - similarity: Cosine similarity score between the two models' coefficients or importances.
    """
    # Extract the weights (coefficients) or feature importances
    def get_model_vector(model):
        if hasattr(model, 'coef_'):  # Linear models with coefficients
            return model.coef_.flatten()
        elif hasattr(model, 'feature_importances_'):  # Tree-based models
            return model.feature_importances_
        else:
            raise ValueError(f"Model of type {type(model)} does not have coefficients or feature importances.")
    
    try:
        # Get vectors for the two models
        vector1 = get_model_vector(model1)
        vector2 = get_model_vector(model2)

        # Ensure vectors are of the same length
        if len(vector1) != len(vector2):
            raise ValueError("Model vectors have different lengths. Ensure the models were trained on the same features.")

        # Calculate cosine similarity
        similarity = cosine_similarity([vector1], [vector2])
        return similarity[0][0]  # Return the scalar similarity value

    except ValueError as e:
        print(f"Error in calculating similarity: {e}")
        return None


# Full pipeline
def pipeline1(X, y, n_features, N_random_state):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y, N_random_state)

    best_model, best_name = model_selection(X_train, y_train)

    selected_features = feature_selection_with_rfe(X_train, y_train, n_features, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy, test_auc = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy

def pipeline2(X, y, N_random_state):
    y = ensure_binary_target(y)
    X_train, X_test, y_train, y_test = split_data(X, y, N_random_state)

    best_model, best_name = model_selection(X_train, y_train)

    
    selected_features, features_number = feature_selection_with_rfe_cv(X_train, y_train, best_model)
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]

    tuned_model = tune_model_hyperparameters(best_model, best_name, X_train_selected, y_train)

    final_model, test_accuracy, test_auc = train_and_evaluate_final_model(
        X_train_selected, y_train, X_test_selected, y_test, tuned_model
    )
    return final_model, selected_features, test_accuracy


def compare_models_and_analyze_topography1(X_data_set1, y_data_set1, X_data_set2, y_data_set2, n_features, N_random_state):
    print("Training data_set1 model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline1(X_data_set1, y_data_set1, n_features, N_random_state)
    X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    y_pred = final_model_data_set1.predict(X_data_set1_selected)
    if test_accuracy_data_set1 < 0.63:
        print(f"Test accuracy ({test_accuracy_data_set1:.4f}) is below 0.63. Stopping pipeline.")
        return None, test_accuracy_data_set1, None, 0, None, 0  # Early exit with placeholder return values
    
    test_accuracy_all = final_model_data_set1.score(X_data_set1_selected, y_data_set1)
    # if test_accuracy_all < 0.9:
    #     print(f"Test accuracy for all ({test_accuracy_all:.4f}) is below 0.9. Stopping pipeline.")
    #     return None, test_accuracy_data_set1, None, None, None, test_accuracy_all
    print("Results for whole data_set1 model:")
    print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
    print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    
    
    print("\nTraining data_set2 model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline1(X_data_set2, y_data_set2, n_features, N_random_state)
    X_data_set2_selected = X_data_set2[:, selected_features_data_set2]
    y_pred2 = final_model_data_set2.predict(X_data_set2_selected)
    if test_accuracy_data_set2 < 0.63:
        print(f"Test accuracy for all ({test_accuracy_all:.4f}) is below 0.9. Stopping pipeline.")
        return None, test_accuracy_data_set1, None, 0, None, test_accuracy_all
    print("Results for whole data_set2 model:")
    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred2))
    

    print("\nEvaluating data_set1 model on data_set2 dataset:")
    X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
    # Check if the model supports probability predictions for AUC computation
    if hasattr(final_model_data_set1, "predict_proba"):
        y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:, 1]  # Probabilities for the positive class
        auc = roc_auc_score(y_data_set2, y_proba_data_set2)
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    else:
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
        auc = None  # AUC not computed due to lack of probability support

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
    try:
        similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")
    return similarity, test_accuracy_data_set1, final_model_data_set1, test_accuracy_data_set2, final_model_data_set2, test_accuracy_all



def compare_models_and_analyze_topography2(X_data_set1, y_data_set1, X_data_set2, y_data_set2, N_random_state):
    print("Training data_set1 model...")
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline2(X_data_set1, y_data_set1, N_random_state)
    # X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    # y_pred = final_model_data_set1.predict(X_data_set1_selected)
    # print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
    # print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    
    print("\nTraining data_set2 model...")
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline2(X_data_set2, y_data_set2, N_random_state)
    # X_data_set2_selected = X_data_set2[:, selected_features_data_set2]
    # y_pred2 = final_model_data_set2.predict(X_data_set2_selected)
    # print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred2))
    # print("Classification Report:\n", classification_report(y_data_set2, y_pred2))
    

    print("\nEvaluating data_set1 model on data_set2 dataset:")
    X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
    # Check if the model supports probability predictions for AUC computation
    if hasattr(final_model_data_set1, "predict_proba"):
        y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:, 1]  # Probabilities for the positive class
        auc = roc_auc_score(y_data_set2, y_proba_data_set2)
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    else:
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
        auc = None  # AUC not computed due to lack of probability support

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
    try:
        similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")
    return similarity, test_accuracy_data_set1

In [4]:
# Define user inputs
user_dir = "/Users/xiaoqianxiao"
project_name = "UKB"
session_ID = 2  # Specify session

In [5]:
# Run the pipeline
data_set = "past_anxiety"  # Dataset identifier
X_pad, df_PAD = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_pad = df_PAD["hospital_not_now"]
y_pad_GAD7 = df_PAD["GAD7_score"]

data_set = "current_anxiety"  # Dataset identifier
X_cad, df_CAD = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_cad = df_CAD["hospital_current_anxiety"]
y_cad_GAD7 = df_CAD["GAD7_score"]

KeyboardInterrupt: 

In [498]:
print(f'n(cad)/(cad_control): {sum(y_cad==True)}/{sum(y_cad==False)} = {((sum(y_cad==True))/(sum(y_cad==False))):.2f}')
print(f'n(pad)/(pad_control): {sum(y_pad==True)}/{sum(y_pad==False)} = {((sum(y_pad==True))/(sum(y_pad==False))):.2f}')

n(cad)/(cad_control): 118/106 = 1.11
n(pad)/(pad_control): 511/478 = 1.07


In [ ]:
n_features = 20
compare_models_and_analyze_topography1(X_cad, y_cad, X_pad, y_pad, n_features)

In [ ]:
similarity, test_accuracy_data_set1 = compare_models_and_analyze_topography2(X_cad, y_cad, X_pad, y_pad)

In [6]:
data_set = "ah"  # Dataset identifier
X_ah, df_ah = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_ah = df_ah["active_history"]

data_set = "ih"  # Dataset identifier
X_ih, df_ih = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_ih = df_ih["inactive_history"]

data_set = "a_noh"  # Dataset identifier
X_a_noh, df_a_noh = process_fMRI_data(data_set, user_dir, project_name, session_ID, dic_cortical_roi, subcortical_index)
y_a_noh = df_a_noh["active_no_history"]

Missing files for subject 1529291, session 2.


In [7]:
print(f'n(ah)/(ah_control): {sum(y_ah==True)}/{sum(y_ah==False)} = {((sum(y_ah==True))/(sum(y_ah==False))):.2f}')
print(f'n(ih)/(ih_control): {sum(y_ih==True)}/{sum(y_ih==False)} = {((sum(y_ih==True))/(sum(y_ih==False))):.2f}')
print(f'n(a_noh)/(a_noh_control): {sum(y_a_noh==True)}/{sum(y_a_noh==False)} = {((sum(y_a_noh==True))/(sum(y_a_noh==False))):.2f}')

n(ah)/(ah_control): 367/337 = 1.09
n(ih)/(ih_control): 874/819 = 1.07
n(a_noh)/(a_noh_control): 250/233 = 1.07


In [ ]:
a = 0
b = 0
N_random_state = 0
while a < 0.65 or b < 0.65:
    n_features = 10
    similarity, test_accuracy_data_set1, final_model_data_set1, test_accuracy_data_set2, final_model_data_set2, test_accuracy_all = compare_models_and_analyze_topography1(X_ah, y_ah, X_ih, y_ih, n_features, N_random_state)
    a = test_accuracy_data_set1
    b = test_accuracy_data_set2
    print(N_random_state)
    N_random_state = N_random_state + 1

In [ ]:
compare_models_and_analyze_topography2(X_ah, y_ah, X_ih, y_ih)

In [ ]:
a = 0
N_random_state = 0
while a < 0.8:
    n_features = 10
    similarity, test_accuracy_data_set1, final_model_data_set1, final_model_data_set2 = compare_models_and_analyze_topography1(X_ah, y_ah, X_a_noh, y_a_noh, n_features,N_random_state)
    a = test_accuracy_data_set1
    print(N_random_state)
    print(test_accuracy_data_set1)
    N_random_state = N_random_state + 1

In [ ]:
compare_models_and_analyze_topography2(X_ah, y_ah, X_a_noh, y_a_noh)

In [29]:
n_features = 10
N_random_state = 0
X_data_set1 = X_ah
y_data_set1 = y_ah
a = 0
while a < 0.65: 
    final_model_data_set1, selected_features_data_set1, test_accuracy_data_set1 = pipeline1(X_data_set1, y_data_set1, n_features, N_random_state)
    X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    y_pred = final_model_data_set1.predict(X_data_set1_selected)
    if test_accuracy_data_set1 > 0.63:
        print("Results for whole data_set1 model:")
        print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
        print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    a = test_accuracy_data_set1
    N_random_state = N_random_state + 1

Model: Logistic Regression, CV Score: 0.5488
Model: Ridge Classifier, CV Score: 0.5488
Model: Lasso (L1), CV Score: 0.5648
Model: LDA, CV Score: 0.4707
Model: Perceptron, CV Score: 0.5204
Model: SVM (Linear), CV Score: 0.5470
Model: Random Forest, CV Score: 0.5542
Best Model: Lasso (L1) with CV score: 0.5648
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Parameters (Broad Search): {'C': 1}
Best Parameters (Narrow Search): {'C': np.float64(2.575)}
Test Set AUC (Final Model): 0.5508
Test Set Accuracy (Final Model): 0.5674
Confusion Matrix:
 [[32 35]
 [26 48]]
Classification Report:
               precision    recall  f1-score   support

           0       0.55      0.48      0.51        67
           1       0.58      0.65      0.61        74

    accuracy                           0.57       141
   macro avg       0.57      0.56

In [56]:
selected_feature_labels = [label for label, selected in zip(connectivity_labels, selected_features_data_set1) if selected]
print(selected_feature_labels)

['lh_dlPFC--THA-DP-lh', 'rh_dlPFC--THA-DP-lh', 'lh_mPFC--lAMY-rh', 'rh_mPFC--THA-DP-lh', 'lh_Ins--THA-DP-lh', 'rh_Ins--THA-DP-lh', 'THA-DP-rh--NAc-shell-rh', 'THA-VP-rh--THA-VA-rh', 'THA-VA-rh--THA-DP-lh', 'THA-DP-lh--THA-VA-lh']


In [53]:
connectivity_labels[selected_features_data_set1]

TypeError: only integer scalar arrays can be converted to a scalar index

In [36]:
n_features = 10
N_random_state = 0
X_data_set1 = X_ih
y_data_set1 = y_ih
a = 0
while a < 0.54: 
    final_model_data_set2, selected_features_data_set2, test_accuracy_data_set2 = pipeline1(X_data_set1, y_data_set1, n_features, N_random_state)
    X_data_set1_selected = X_data_set1[:, selected_features_data_set1]
    y_pred = final_model_data_set1.predict(X_data_set1_selected)
    if test_accuracy_data_set1 >= 0.53:
        print("Results for whole data_set1 model:")
        print("Confusion Matrix:\n", confusion_matrix(y_data_set1, y_pred))
        print("Classification Report:\n", classification_report(y_data_set1, y_pred))
    a = test_accuracy_data_set2
    N_random_state = N_random_state + 1

Model: Logistic Regression, CV Score: 0.4793
Model: Ridge Classifier, CV Score: 0.4800
Model: Lasso (L1), CV Score: 0.4786
Model: LDA, CV Score: 0.4874
Model: Perceptron, CV Score: 0.4911
Model: SVM (Linear), CV Score: 0.4807
Model: Random Forest, CV Score: 0.5022
Best Model: Random Forest with CV score: 0.5022
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 3 candidates, totalling 30 fits
Best Parameters (Broad Search): {'max_depth': None, 'n_estimators': 200}
Best Parameters (Narrow Search): {'max_depth': None, 'n_estimators': 250}
Test Set AUC (Final Model): 0.4962
Test Set Accuracy (Final Model): 0.4926
Confusion Matrix:
 [[77 87]
 [85 90]]
Classification Report:
               precision    recall  f1-score   support

           0       0.48      0.47      0.47       164
           1       0.51      0.51      0.51       175

    accuracy                      

/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/xiaoqianxiao/Library/Caches/pypoetry/virtualenvs/ukbnexus-xiaoqian-PL9ZUpYW-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined

Model: Ridge Classifier, CV Score: 0.5325
Model: Lasso (L1), CV Score: 0.5347
Model: LDA, CV Score: 0.5126
Model: Perceptron, CV Score: 0.5200
Model: SVM (Linear), CV Score: 0.5251
Model: Random Forest, CV Score: 0.4927
Best Model: Lasso (L1) with CV score: 0.5347
Model lacks coefficients/feature importance; using univariate feature selection.
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Parameters (Broad Search): {'C': 1}
Best Parameters (Narrow Search): {'C': np.float64(2.575)}
Test Set AUC (Final Model): 0.4869
Test Set Accuracy (Final Model): 0.4838
Confusion Matrix:
 [[ 47 117]
 [ 58 117]]
Classification Report:
               precision    recall  f1-score   support

           0       0.45      0.29      0.35       164
           1       0.50      0.67      0.57       175

    accuracy                           0.48       339
   macro avg       0.47      0.48      0.46       339
weighted avg       0

In [39]:
X_data_set1 = X_ah
y_data_set1 = y_ah
X_data_set2 = X_ih
y_data_set2 = y_ih
X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
# Check if the model supports probability predictions for AUC computation
if hasattr(final_model_data_set1, "predict_proba"):
    y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:,
                        1]  # Probabilities for the positive class
    auc = roc_auc_score(y_data_set2, y_proba_data_set2)
    print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
else:
    print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
    auc = None  # AUC not computed due to lack of probability support
print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
try:
    similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
    print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
except ValueError as e:
    similarity = "N/A (Model type not compatible for cosine similarity)"
    print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")

Accuracy of data_set1 model on data_set2 data: 0.5251
Dataset 2 - Accuracy: 0.5251, AUC: 0.5183

Calculating cosine similarity between data_set1 and data_set2 model weights:
Cosine similarity between data_set1 and data_set2 model weights: 0.1477


In [ ]:
X_data_set2_selected = X_data_set2[:, selected_features_data_set1]
    y_pred_data_set2 = final_model_data_set1.predict(X_data_set2_selected)
    accuracy = accuracy_score(y_data_set2, y_pred_data_set2)
    print(f"Accuracy of data_set1 model on data_set2 data: {accuracy:.4f}")
    # Check if the model supports probability predictions for AUC computation
    if hasattr(final_model_data_set1, "predict_proba"):
        y_proba_data_set2 = final_model_data_set1.predict_proba(X_data_set2_selected)[:, 1]  # Probabilities for the positive class
        auc = roc_auc_score(y_data_set2, y_proba_data_set2)
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}, AUC: {auc:.4f}")
    else:
        print(f"Dataset 2 - Accuracy: {accuracy:.4f}")
        auc = None  # AUC not computed due to lack of probability support

    print("Confusion Matrix:\n", confusion_matrix(y_data_set2, y_pred_data_set2))
    print("Classification Report:\n", classification_report(y_data_set2, y_pred_data_set2))

    print("\nCalculating cosine similarity between data_set1 and data_set2 model weights:")
    try:
        similarity = calculate_model_similarity(final_model_data_set1, final_model_data_set2)
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity:.4f}")
    except ValueError as e:
        similarity = "N/A (Model type not compatible for cosine similarity)"
        print(f"Cosine similarity between data_set1 and data_set2 model weights: {similarity}")
    return similarity, test_accuracy_data_set1, final_model_data_set1, test_accuracy_data_set2, final_model_data_set2, test_accuracy_all